### Exercise 3 - Solution

You are given three JSON files, representing a default set of settings, and environment specific settings.
The files are included in the downloads, and are named:
* `common.json`
* `dev.json`
* `prod.json`

Your goal is to write a function that has a single argument, the environment name, and returns the "combined" dictionary that merges the two dictionaries together, with the environment specific settings overriding any common settings already defined.

For simplicity, assume that the argument values are going to be the same as the file names, without the `.json` extension. So for example, `dev` or `prod`.

The wrinkle: We don't want to duplicate data for the "merged" dictionary - use `ChainMap` to implement this instead.

In [5]:
import json

def load_settings(env):
    with open(f'{env}.json') as f:
        settings = json.load(f)
    return settings

In [4]:
from pprint import pprint

In [6]:
del globals('load_setting')

SyntaxError: cannot delete function call (1461479435.py, line 1)

In [7]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  "import json\n\ndef load_setting(env):\n    with open(f'{env}.json') as f:\n        settings = jeson.load(f)\n    return settings",
  'from pprint import pprint',
  "import json\n\ndef load_setting(env):\n    with open(f'{env}.json') as f:\n        settings = json.load(f)\n    return settings",
  'from pprint import pprint',
  "import json\n\ndef load_settings(env):\n    with open(f'{env}.json') as f:\n        settings = json.load(f)\n    return settings",
  "del globals('load_setting')",
  'globals()'],
 '_oh': {},
 '_dh': [PosixPath('/workspaces/LocalDev/learning/python_deepdive_3/10_coding_exercises_3')],
 'In': ['',
  "import json\n\ndef load_setting(env):\n    with open(f'{env}.json') as f:\n        setting